## Row reduction and LU Decomposition

In [37]:
import numpy as np
import sympy as sym
import scipy.linalg as sc

Solve a system of equations with a matrix system with Python.

The order is important, because in matrix multiplication, order matters.


$$A=\begin{vmatrix}1 & 1\\
2 & 4
\end{vmatrix}$$


$$X=\begin{vmatrix}a\\b
\end{vmatrix}$$


$$B=\begin{vmatrix}35\\
94
\end{vmatrix}$$

AX = B \
A<sup>-1</sup>AX = A<sup>-1</sup>B \
X = A<sup>-1</sup>B

to solve for x, pre-multiply the inverse of A by B

In [38]:
A = np.array([[1,1],[2,4]])
B = np.array([[35], [94]])

# solve for x
X = np.linalg.inv(A)@B
print(X)

[[23.]
 [12.]]


### Row reduction

Row reduction means iteratively apply two operations-scalar multiplication and addition, to rows of a matrix. Row reduction relies on the same principle as adding equations to other equations within a system.

<i>The goal of row reduction is to transform a dense matrix into an upper-triangular matrix.</i> \
The upper-triangular matrix that results from row reduction is called the <i>echelon form</i> of the matrix

As the matrix before and after the row reduction is linked by a linear transformation, it is possible to express row reduction with a matrix  \
As there are many (or infinite) row echelons form for the same matrix, two of them that are unique, reduced row echelon form and U are the most important ones.

There is no direct function in Pyhon to calculate a row echelon form of a matrix

### Gaussian elimination

It is possible to solve a matrix system of equation for X without using inversion. This method is called Gaussian Elimination.

Gaussian elimination consists in:
- augment the matrix of coefficients by the vector of constants
- row reduce to echelon form
- use back substitution to solve for each variable in turn

It exists also the Gaussian-Jordan elimination, which keep on reducing upward to eliminate all the elements above each pivot. In other words, it transform the row echelon matrix in a matrix that, apart from the last column, have only ones and zero on each echelon. This form is called <b>reduced row echelon form</b> and has been used for centuries to solve system of equations manually.

The reduced row echelon form, or RREF, is unique for a matrix, The Sympy library of Python has a function to calculate RREF.


In [39]:
# a numpy array converted in sympy
M = np.array([[1,1,4],[-1/2,1,2]])
SymMat = sym.Matrix(M)

print(SymMat)
print('reduced row echelon form:')
print(SymMat.rref()[0])

Matrix([[1.00000000000000, 1.00000000000000, 4.00000000000000], [-0.500000000000000, 1.00000000000000, 2.00000000000000]])
reduced row echelon form:
Matrix([[1, 0, 1.33333333333333], [0, 1, 2.66666666666667]])


Consider this equation:

$$\begin{vmatrix}a & b\\
c & d
\end{vmatrix} . \begin{vmatrix}x\\
y
\end{vmatrix} = \begin{vmatrix}1\\
0
\end{vmatrix}$$

The constant vector is the first column of the 2x2 identity matrix. That means that applying RREF to a square full-rank matrix augmented by the first column of the identity matrix wll reveal the linear transformation that brings the matrix into the first column of the identity matrix. And that in turn, that the vector [x,y]<sup>T</sup> is the first column of the matrix inverse.

If we repeat this operation having on the left the other  columns of the identity matrix, we find that infact, every time the [x,y] vector will be the transpose of the correspondant column of M inverse.

And that means, it is possible to obtain the entire inverse matrix by augmenting the entire original matrix by the identity matrix and solve one single RREF.

RREF([A | I]) = [I | A<sup>-1</sup>]

This is interesting because it provides a mechanism for computing the matrix inverse without computing determinants. On the other hand, row reduction involves a lotof division, which increases the risk of numerical precision errors. The Gaussian-Jordan elimination provides a more numerically stable way to compute the matrix than the full original algorithm, but a matrix that is close to singular will be difficul to invert, regardless of the used algorithm...

In [40]:
# given a squared full-rank matrix:
M = np.array([[1,2],[5,6]])

# this is the inverse calculated by numpy
print(np.linalg.inv(M))

# augment the original matrix by an identity matrix about the same shape and solve one RREF
id = np.eye(M.shape[0])
augM = np.hstack([M,id])
print(augM)

SymM = sym.Matrix(augM)
print(SymM)

RREF = SymM.rref()[0]
print(RREF)

# the last two columns are the inverse of the original matrix
inv_by_RREF = np.array(RREF).astype(np.float64)[:,-2:]
print(inv_by_RREF)

[[-1.5   0.5 ]
 [ 1.25 -0.25]]
[[1. 2. 1. 0.]
 [5. 6. 0. 1.]]
Matrix([[1.00000000000000, 2.00000000000000, 1.00000000000000, 0], [5.00000000000000, 6.00000000000000, 0, 1.00000000000000]])
Matrix([[1, 0, -1.50000000000000, 0.500000000000000], [0, 1, 1.25000000000000, -0.250000000000000]])
[[-1.5   0.5 ]
 [ 1.25 -0.25]]


### LU Decomposition

LU means "lower triangular" and "upper triangular"
The idea is to decompose a matrix into the product of two triangular matrices.

A = LU

Here is a numerical example:

$$\begin{vmatrix}2 & 2 & 4\\
1 & 0 & 3 \\
2 & 1 & 2
\end{vmatrix} = \begin{vmatrix}1 & 0 & 0\\
1/2 & 1 & 0 \\
1 & 1 & 1
\end{vmatrix} . \begin{vmatrix}2 & 2 & 4\\
0 & -1 & 1 \\
0 & 0 & -3
\end{vmatrix}$$

These two matrices comes from the definition of row reduction:

L<sup>-1</sup>A = U

U = echelon form

LU decoposition is not unique: there is an infinite number of lower- and upper-triangular matrices that could multiply to produce a matrix. However, adding the constraint of the diagonal of L = 1 ensures that the LU decomposition is unique for a full-rank squared matrix

In [43]:
A = np.array([[2,2,4],[1,0,3],[2,1,2]])

_,L,U = sc.lu(A)

print('L: ')
print(L)
print('U: ')
print(U)

L: 
[[1.  0.  0. ]
 [0.5 1.  0. ]
 [1.  1.  1. ]]
U: 
[[ 2.  2.  4.]
 [ 0. -1.  1.]
 [ 0.  0. -3.]]


### Row swapping via Permutation Matrices

Permutation matrices are often labeled with P. They are used to swap position of a row of a matrix to facilitate row reduction.

The permutation matrices are orthogonal, means that P<sup>-1</sup>= P<sup>T</sup>
The reason is that all elements in a permutation matrix are either 0 or 1, and because rows are swapped only once, each column has exactly one non zero element (permutation matrices leave the original values intact, so that means that they are all identity matrix with swapped rows).

In [45]:
M = np.array([[3,2,1],[0,0,5],[0,7,2]])

P = np.array([[0,0,1],[1,0,0],[0,1,0]])

Mp = P@M

print('M:')
print(M)
print('P:')
print(P)
print('row swapped M:')
print(Mp)

M:
[[3 2 1]
 [0 0 5]
 [0 7 2]]
P:
[[0 0 1]
 [1 0 0]
 [0 1 0]]
row swapped M:
[[0 7 2]
 [3 2 1]
 [0 0 5]]
